In [9]:
import numpy as np
import matplotlib.pyplot as plt
from renormalization_np import track_rg_flow
from tabulate import tabulate

def generate_rg_flow(J, M, n_max=20, n_steps=50, b=2, d=2):
    """
    Generate a single RG flow at specific J and M values
    
    Args:
        J: Two-spin coupling
        M: Four-spin coupling
        n_max: Maximum Fourier mode
        n_steps: Number of RG steps
        b: Length rescaling factor
        d: Dimension
        
    Returns:
        Final coefficient matrix at fixed point (normalized)
    """
    # Track the RG flow
    flow_history = track_rg_flow(J, J, M, b, d, n_max, n_steps)
    
    # Get the final matrix
    final_matrix = flow_history[-1]
    
    # Normalize by (0,0) coefficient
    center_idx = n_max
    if abs(final_matrix[center_idx, center_idx]) > 1e-10:
        final_matrix = final_matrix / final_matrix[center_idx, center_idx]
    
    return final_matrix

def calculate_villain_coefficients(Jv, coeff_indices, n_max=20):
    """
    Calculate the Villain fixed-line coefficients for a given Jv
    
    Args:
        Jv: Villain coupling parameter
        coeff_indices: List of (n,m) tuples for desired coefficients
        n_max: Maximum Fourier mode
        
    Returns:
        Dictionary mapping (n,m) tuples to Villain coefficient values
    """
    villain_coeffs = {}
    
    for n, m in coeff_indices:
        # Calculate exponent based on coefficient indices
        exponent = n**2 + m**2
        
        # Calculate Villain coefficient
        villain_coeffs[(n, m)] = np.exp(-exponent/(2*abs(Jv)))
        
        # For odd indices and negative Jv, coefficients may be negative
        if (n + m) % 2 == 1 and Jv < 0:
            villain_coeffs[(n, m)] *= -1
    
    return villain_coeffs

def estimate_Jv_from_coefficients(final_matrix, n_max=20):
    """
    Estimate Jv from coefficient matrix using the ratio method
    
    Args:
        final_matrix: Normalized coefficient matrix from RG flow
        n_max: Maximum Fourier mode
        
    Returns:
        Estimated Jv value
    """
    # Extract key coefficients for estimation
    center_idx = n_max
    lambda_10 = final_matrix[center_idx+1, center_idx].real
    lambda_20 = final_matrix[center_idx+2, center_idx].real
    
    # Only proceed if both coefficients are non-zero
    if abs(lambda_10) > 1e-10 and abs(lambda_20) > 1e-10:
        # Calculate ratio and estimate Jv
        ratio = abs(lambda_20 / lambda_10)
        
        # For normal decay, λ₍₂,₀₎ should be smaller than λ₍₁,₀₎
        if ratio < 1:
            Jv = -3 / (2 * np.log(ratio))  # From the ratio of λ₍₂,₀₎/λ₍₁,₀₎ = exp(-3/2Jv)
        else:
            # For inverted cases, adjust the formula
            Jv = 3 / (2 * np.log(ratio))
            
        return Jv
    
    return None

def create_coefficient_comparison_table(J, M, n_max=20, n_steps=50, b=2, d=2, decimals=6):
    """
    Generate a table comparing RG flow coefficients with Villain fixed-line values
    
    Args:
        J: Two-spin coupling
        M: Four-spin coupling
        n_max: Maximum Fourier mode
        n_steps: Number of RG steps
        b: Length rescaling factor
        d: Dimension
        decimals: Number of decimal places to display
        
    Returns:
        Table as string
    """
    # Generate RG flow and get final coefficient matrix
    final_matrix = generate_rg_flow(J, M, n_max, n_steps, b, d)
    
    # Define unique coefficients to display
    # Use a list of tuples for consistent ordering
    center_idx = n_max
    coeff_indices = [
        (0, 0),   # (0,0)
        (1, 0),   # (1,0)
        (1, 1),   # (1,1)
        (2, 0),   # (2,0)
        (2, 1),   # (2,1)
        (2, 2),   # (2,2)
        (3, 0),   # (3,0)
        (3, 3),   # (3,3)
        (4, 0),   # (4,0)
        (4, 4)    # (4,4)
    ]
    
    # Extract coefficients from the matrix
    rg_coeffs = {}
    for n, m in coeff_indices:
        n_idx = center_idx + n
        m_idx = center_idx + m
        if 0 <= n_idx < final_matrix.shape[0] and 0 <= m_idx < final_matrix.shape[1]:
            rg_coeffs[(n, m)] = final_matrix[n_idx, m_idx].real
    
    # Estimate Jv from the coefficient matrix
    estimated_Jv = estimate_Jv_from_coefficients(final_matrix, n_max)
    
    # Calculate Villain fixed-line coefficients
    villain_coeffs = {}
    if estimated_Jv is not None:
        villain_coeffs = calculate_villain_coefficients(estimated_Jv, coeff_indices, n_max)
    
    # Create table headers
    headers = ["Source", "λ(0,0)", "λ(1,0)", "λ(1,1)", "λ(2,0)", "λ(2,1)", 
               "λ(2,2)", "λ(3,0)", "λ(3,3)", "λ(4,0)", "λ(4,4)"]
    
    # Create table rows
    rg_row = ["RG Flow"]
    villain_row = ["Villain (Jv={:.3f})".format(estimated_Jv if estimated_Jv is not None else 0)]
    
    # Fill in coefficient values
    for n, m in coeff_indices:
        # Add RG coefficient
        if (n, m) in rg_coeffs:
            rg_row.append("{:.{prec}f}".format(rg_coeffs[(n, m)], prec=decimals))
        else:
            rg_row.append("N/A")
        
        # Add Villain coefficient
        if (n, m) in villain_coeffs:
            villain_row.append("{:.{prec}f}".format(villain_coeffs[(n, m)], prec=decimals))
        else:
            villain_row.append("N/A")
    
    # Create table
    table_data = [rg_row, villain_row]
    table = tabulate(table_data, headers=headers, tablefmt="grid")
    
    # Print parameters
    param_info = f"Parameters: J={J}, M={M}, n_steps={n_steps}, b={b}, d={d}\n"
    full_table = param_info + table
    
    return full_table


In [25]:
table = create_coefficient_comparison_table(J=1.5, M=0.5, n_max=20, n_steps=10, b=3, d=2, decimals=3)
print(table)

Ferromagnetic Phase Example:




100%|██████████| 10/10 [00:08<00:00,  1.12it/s]

Parameters: J=1.5, M=0.5, n_steps=10, b=3, d=2
+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| Source             |   λ(0,0) |   λ(1,0) |   λ(1,1) |   λ(2,0) |   λ(2,1) |   λ(2,2) |   λ(3,0) |   λ(3,3) |   λ(4,0) |   λ(4,4) |
+====================+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+
| RG Flow            |        1 |    0.837 |      0.7 |    0.491 |    0.411 |    0.241 |    0.201 |    0.041 |    0.058 |    0.003 |
+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| Villain (Jv=2.808) |        1 |    0.837 |      0.7 |    0.491 |    0.411 |    0.241 |    0.201 |    0.041 |    0.058 |    0.003 |
+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+


In [29]:
import numpy as np
import matplotlib.pyplot as plt
from renormalization_np import track_rg_flow
from tabulate import tabulate

def generate_rg_flow(J, M, n_max=20, n_steps=50, b=2, d=2):
    """
    Generate a single RG flow at specific J and M values
    
    Args:
        J: Two-spin coupling
        M: Four-spin coupling
        n_max: Maximum Fourier mode
        n_steps: Number of RG steps
        b: Length rescaling factor
        d: Dimension
        
    Returns:
        Final coefficient matrix at fixed point (normalized)
    """
    # Track the RG flow
    flow_history = track_rg_flow(J, J, M, b, d, n_max, n_steps)
    
    # Get the final matrix
    final_matrix = flow_history[-1]
    
    # Normalize by (0,0) coefficient
    center_idx = n_max
    if abs(final_matrix[center_idx, center_idx]) > 1e-10:
        final_matrix = final_matrix / final_matrix[center_idx, center_idx]
    
    return final_matrix

def calculate_villain_coefficients(Jv, coeff_indices, n_max=20):
    """
    Calculate the Villain fixed-line coefficients for a given Jv
    
    Args:
        Jv: Villain coupling parameter
        coeff_indices: List of (n,m) tuples for desired coefficients
        n_max: Maximum Fourier mode
        
    Returns:
        Dictionary mapping (n,m) tuples to Villain coefficient values
    """
    villain_coeffs = {}
    
    for n, m in coeff_indices:
        # Calculate exponent based on coefficient indices
        exponent = n**2 + m**2
        
        # Calculate Villain coefficient
        villain_coeffs[(n, m)] = np.exp(-exponent/(2*abs(Jv)))
        
        # For odd indices and negative Jv, coefficients may be negative
        if (n + m) % 2 == 1 and Jv < 0:
            villain_coeffs[(n, m)] *= -1
    
    return villain_coeffs

def estimate_Jv_from_coefficients(final_matrix, n_max=20):
    """
    Estimate Jv from coefficient matrix using the ratio method
    
    Args:
        final_matrix: Normalized coefficient matrix from RG flow
        n_max: Maximum Fourier mode
        
    Returns:
        Estimated Jv value
    """
    # Extract key coefficients for estimation
    center_idx = n_max
    lambda_10 = final_matrix[center_idx+1, center_idx].real
    lambda_20 = final_matrix[center_idx+2, center_idx].real
    
    # Only proceed if both coefficients are non-zero
    if abs(lambda_10) > 1e-10 and abs(lambda_20) > 1e-10:
        # Calculate ratio and estimate Jv
        ratio = abs(lambda_20 / lambda_10)
        
        # For normal decay, λ₍₂,₀₎ should be smaller than λ₍₁,₀₎
        if ratio < 1:
            Jv = -3 / (2 * np.log(ratio))  # From the ratio of λ₍₂,₀₎/λ₍₁,₀₎ = exp(-3/2Jv)
        else:
            # For inverted cases, adjust the formula
            Jv = 3 / (2 * np.log(ratio))
            
        return Jv
    
    return None

def generate_coefficient_indices(n_max, exclude_center=True):
    """
    Generate a list of unique coefficient indices to display
    
    Args:
        n_max: Maximum Fourier mode
        exclude_center: Whether to exclude the (0,0) coefficient
        
    Returns:
        List of (n,m) tuples
    """
    indices = []
    
    # Generate diagonal terms (n,n)
    for n in range(0, n_max+1):
        if n > 0 or not exclude_center:  # Skip (0,0) if exclude_center is True
            indices.append((n, n))
    
    # Generate (n,0) terms
    for n in range(1, n_max+1):
        indices.append((n, 0))
    
    # Generate (0,m) terms - but only if m > n for uniqueness
    # (we don't need both (n,0) and (0,n) due to symmetry)
    
    # Generate other important off-diagonal terms (n,m)
    # Including some key off-diagonal terms that might be interesting
    for n in range(1, n_max):
        for m in range(n+1, n_max+1):
            # We'll include terms where n and m have small values
            # to limit the number of terms
            if n + m <= n_max + 1:  # A heuristic to limit the number of terms
                indices.append((n, m))
    
    # Sort indices by total magnitude (n+m) and then by n
    indices.sort(key=lambda x: (x[0] + x[1], x[0]))
    
    return indices

def create_coefficient_comparison_table(J, M, n_max=4, n_steps=50, b=2, d=2, decimals=6):
    """
    Generate a table comparing RG flow coefficients with Villain fixed-line values
    
    Args:
        J: Two-spin coupling
        M: Four-spin coupling
        n_max: Maximum Fourier mode
        n_steps: Number of RG steps
        b: Length rescaling factor
        d: Dimension
        decimals: Number of decimal places to display
        
    Returns:
        Table as string
    """
    # Generate RG flow and get final coefficient matrix
    final_matrix = generate_rg_flow(J, M, n_max=n_max*2, n_steps=n_steps, b=b, d=d)
    
    # Define unique coefficients to display
    coeff_indices = generate_coefficient_indices(n_max)
    
    # Extract coefficients from the matrix
    center_idx = final_matrix.shape[0] // 2  # Center index of the full matrix
    rg_coeffs = {}
    for n, m in coeff_indices:
        n_idx = center_idx + n
        m_idx = center_idx + m
        if 0 <= n_idx < final_matrix.shape[0] and 0 <= m_idx < final_matrix.shape[1]:
            rg_coeffs[(n, m)] = final_matrix[n_idx, m_idx].real
    
    # Estimate Jv from the coefficient matrix
    estimated_Jv = estimate_Jv_from_coefficients(final_matrix, center_idx)
    
    # Calculate Villain fixed-line coefficients
    villain_coeffs = {}
    if estimated_Jv is not None:
        villain_coeffs = calculate_villain_coefficients(estimated_Jv, coeff_indices, center_idx)
    
    # Create table headers
    headers = ["Source"] + [f"λ({n},{m})" for n, m in coeff_indices]
    
    # Create table rows
    rg_row = ["RG Flow"]
    villain_row = ["Villain (Jv={:.4f})".format(estimated_Jv if estimated_Jv is not None else 0)]
    
    # Fill in coefficient values
    for n, m in coeff_indices:
        # Add RG coefficient
        if (n, m) in rg_coeffs:
            rg_row.append("{:.{prec}f}".format(rg_coeffs[(n, m)], prec=decimals))
        else:
            rg_row.append("N/A")
        
        # Add Villain coefficient
        if (n, m) in villain_coeffs:
            villain_row.append("{:.{prec}f}".format(villain_coeffs[(n, m)], prec=decimals))
        else:
            villain_row.append("N/A")
    
    # Create table
    table_data = [rg_row, villain_row]
    table = tabulate(table_data, headers=headers, tablefmt="grid")
    
    # Print parameters
    param_info = f"Parameters: J={J}, M={M}, n_steps={n_steps}, b={b}, d={d}\n"
    full_table = param_info + table
    
    return full_table

In [47]:
ferromagnetic_table = create_coefficient_comparison_table(J=2, M=0, n_max=5, n_steps=10, b=3, d=2, decimals=3)
print(ferromagnetic_table)

100%|██████████| 10/10 [00:00<00:00, 16.20it/s]

Parameters: J=2, M=0, n_steps=10, b=3, d=2
+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| Source              |   λ(1,0) |   λ(1,1) |   λ(2,0) |   λ(1,2) |   λ(3,0) |   λ(1,3) |   λ(2,2) |   λ(4,0) |   λ(1,4) |   λ(2,3) |   λ(5,0) |   λ(1,5) |   λ(2,4) |   λ(3,3) |   λ(4,4) |   λ(5,5) |
+=====================+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+
| RG Flow             |    0.827 |    0.684 |    0.468 |    0.387 |    0.181 |    0.149 |    0.219 |    0.048 |     0.04 |    0.085 |    0.009 |    0.007 |    0.022 |    0.033 |    0.002 |        0 |
+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+--

In [88]:
import numpy as np
import matplotlib.pyplot as plt
from renormalization_np import track_rg_flow
from tabulate import tabulate

def generate_rg_flow(J, M, n_max=20, n_steps=50, b=2, d=2):
    """
    Generate a single RG flow at specific J and M values
    
    Args:
        J: Two-spin coupling
        M: Four-spin coupling
        n_max: Maximum Fourier mode
        n_steps: Number of RG steps
        b: Length rescaling factor
        d: Dimension
        
    Returns:
        Final coefficient matrix at fixed point (normalized)
    """
    # Track the RG flow
    flow_history = track_rg_flow(J, J, M, b, d, n_max, n_steps)
    
    # Get the final matrix
    final_matrix = flow_history[-1]
    
    # Normalize by (0,0) coefficient
    center_idx = n_max
    if abs(final_matrix[center_idx, center_idx]) > 1e-10:
        final_matrix = final_matrix / final_matrix[center_idx, center_idx]
    
    return final_matrix

def calculate_villain_coefficients(Jv, indices, single_index=False):
    """
    Calculate the Villain fixed-line coefficients for a given Jv
    
    Args:
        Jv: Villain coupling parameter
        indices: List of indices (either n or (n,m) depending on single_index)
        single_index: Whether to use single index notation (for M=0 case)
        
    Returns:
        Dictionary mapping indices to Villain coefficient values
    """
    villain_coeffs = {}
    
    if single_index:
        # Single index case for M=0 (factorized XY model)
        for n in indices:
            # Calculate exponent based on single index
            exponent = n**2
            
            # Calculate Villain coefficient
            villain_coeffs[n] = np.exp(-exponent/(2*abs(Jv)))
            
            # For odd indices and negative Jv, coefficients may be negative
            if n % 2 == 1 and Jv < 0:
                villain_coeffs[n] *= -1
    else:
        # Double index case for general Ashkin-Teller model
        for n, m in indices:
            # Calculate exponent based on coefficient indices
            exponent = n**2 + m**2
            
            # Calculate Villain coefficient
            villain_coeffs[(n, m)] = np.exp(-exponent/(2*abs(Jv)))
            
            # For odd indices and negative Jv, coefficients may be negative
            if (n + m) % 2 == 1 and Jv < 0:
                villain_coeffs[(n, m)] *= -1
    
    return villain_coeffs

def estimate_Jv_from_coefficients(final_matrix, n_max=20, M=0):
    """
    Estimate Jv from coefficient matrix using the ratio method
    
    Args:
        final_matrix: Normalized coefficient matrix from RG flow
        n_max: Maximum Fourier mode
        M: Four-spin coupling (to determine if we're in M=0 case)
        
    Returns:
        Estimated Jv value
    """
    center_idx = n_max
    
    if abs(M) < 1e-10:
        # M=0 case: Use the single index approach
        lambda_1 = final_matrix[center_idx+1, center_idx].real
        lambda_2 = final_matrix[center_idx+2, center_idx].real
    else:
        # General case: Use the double index approach
        lambda_10 = final_matrix[center_idx+1, center_idx].real
        lambda_20 = final_matrix[center_idx+2, center_idx].real
        
        # Use lambda_10 and lambda_20 as our reference
        lambda_1 = lambda_10
        lambda_2 = lambda_20
    
    # Only proceed if both coefficients are non-zero
    if abs(lambda_1) > 1e-10 and abs(lambda_2) > 1e-10:
        # Calculate ratio and estimate Jv
        ratio = abs(lambda_2 / lambda_1)
        
        # For normal decay, λ₂ should be smaller than λ₁
        if ratio < 1:
            Jv = -3 / (2 * np.log(ratio))  # From the ratio of λ₂/λ₁ = exp(-3/2Jv)
        else:
            # For inverted cases, adjust the formula
            Jv = 3 / (2 * np.log(ratio))
            
        return Jv
    
    return None

def extract_factorized_coefficients(final_matrix, n_max, max_index=10):
    """
    Extract coefficients from a 2D matrix using single index notation for M=0 case
    
    Args:
        final_matrix: Normalized coefficient matrix from RG flow
        n_max: Maximum Fourier mode in the matrix
        max_index: Maximum single index to extract
        
    Returns:
        Dictionary mapping single indices to coefficient values
    """
    center_idx = n_max
    coeffs = {}
    
    # Extract coefficients λ₁, λ₂, ..., λₙ along the first axis
    for n in range(1, max_index + 1):
        if center_idx + n < final_matrix.shape[0]:
            coeffs[n] = final_matrix[center_idx + n, center_idx].real
    
    return coeffs

def create_coefficient_comparison_table(J, M, n_max=4, n_steps=50, b=2, d=2, decimals=6):
    """
    Generate a table comparing RG flow coefficients with Villain fixed-line values
    
    Args:
        J: Two-spin coupling
        M: Four-spin coupling
        n_max: Maximum Fourier mode
        n_steps: Number of RG steps
        b: Length rescaling factor
        d: Dimension
        decimals: Number of decimal places to display
        
    Returns:
        Table as string
    """
    # Generate RG flow and get final coefficient matrix
    final_matrix = generate_rg_flow(J, M, n_max=n_max*2, n_steps=n_steps, b=b, d=d)
    center_idx = final_matrix.shape[0] // 2  # Center index of the full matrix
    
    # Check if we're in the M=0 case (factorized XY model)
    is_factorized = abs(M) < 1e-10
    
    if is_factorized:
        # For M=0, extract coefficients using single index notation
        max_single_index = n_max #min(10, n_max)  # Limit to first 10 coefficients
        indices = list(range(1, max_single_index + 1))  # λ₁, λ₂, ..., λₙ
        rg_coeffs = extract_factorized_coefficients(final_matrix, center_idx, max_single_index)
        
        # Create table headers
        headers = ["Source"] + [f"λ({n})" for n in indices]
        
        # Estimate Jv from the coefficient matrix
        estimated_Jv = estimate_Jv_from_coefficients(final_matrix, center_idx, M)
        
        # Calculate Villain fixed-line coefficients
        villain_coeffs = {}
        if estimated_Jv is not None:
            villain_coeffs = calculate_villain_coefficients(estimated_Jv, indices, single_index=True)
        
        # Create table rows
        rg_row = ["RG Flow"]
        villain_row = ["Villain (Jv={:.4f})".format(estimated_Jv if estimated_Jv is not None else 0)]
        
        # Fill in coefficient values
        for n in indices:
            # Add RG coefficient
            if n in rg_coeffs:
                rg_row.append("{:.{prec}f}".format(rg_coeffs[n], prec=decimals))
            else:
                rg_row.append("N/A")
            
            # Add Villain coefficient
            if n in villain_coeffs:
                villain_row.append("{:.{prec}f}".format(villain_coeffs[n], prec=decimals))
            else:
                villain_row.append("N/A")
    else:
        # For general case, use the (n,m) notation
        # Define unique coefficients to display
        coeff_indices = []
        
        # Include key diagonal and off-diagonal terms
        for n in range(1, n_max + 1):
            coeff_indices.append((n, 0))  # (n,0) terms
            coeff_indices.append((0, n))  # (0,n) terms
            if n <= n_max//2:
                coeff_indices.append((n, n))  # Diagonal (n,n) up to n_max/2
                
        # Include a few more important terms to showcase phase characteristics
        if n_max >= 2:
            coeff_indices.append((1, 1))  # (1,1) is important for all phases
            coeff_indices.append((2, 1))  # (2,1) shows asymmetry
            
        # Sort by total index value
        coeff_indices.sort(key=lambda x: (x[0] + x[1], x[0]))
        
        # Remove duplicates if any
        coeff_indices = list(dict.fromkeys(coeff_indices))
        
        # Extract coefficients from the matrix
        rg_coeffs = {}
        for n, m in coeff_indices:
            n_idx = center_idx + n
            m_idx = center_idx + m
            if 0 <= n_idx < final_matrix.shape[0] and 0 <= m_idx < final_matrix.shape[1]:
                rg_coeffs[(n, m)] = final_matrix[n_idx, m_idx].real
        
        # Create table headers
        headers = ["Source"] + [f"λ₍{n},{m}₎" for n, m in coeff_indices]
        
        # Estimate Jv from the coefficient matrix
        estimated_Jv = estimate_Jv_from_coefficients(final_matrix, center_idx, M)
        
        # Calculate Villain fixed-line coefficients
        villain_coeffs = {}
        if estimated_Jv is not None:
            villain_coeffs = calculate_villain_coefficients(estimated_Jv, coeff_indices)
        
        # Create table rows
        rg_row = ["RG Flow"]
        villain_row = ["Villain (Jv={:.4f})".format(estimated_Jv if estimated_Jv is not None else 0)]
        
        # Fill in coefficient values
        for n, m in coeff_indices:
            # Add RG coefficient
            if (n, m) in rg_coeffs:
                rg_row.append("{:.{prec}f}".format(rg_coeffs[(n, m)], prec=decimals))
            else:
                rg_row.append("N/A")
            
            # Add Villain coefficient
            if (n, m) in villain_coeffs:
                villain_row.append("{:.{prec}f}".format(villain_coeffs[(n, m)], prec=decimals))
            else:
                villain_row.append("N/A")
    
    # Create table
    table_data = [rg_row, villain_row]
    table = tabulate(table_data, headers=headers, tablefmt="grid")
    
    # Print parameters and model type
    model_type = "XY Model (Factorized)" if is_factorized else "Continuous Ashkin-Teller Model"
    param_info = f"Parameters: J={J}, M={M}, n_steps={n_steps}, b={b}, d={d}\nModel: {model_type}\n"
    full_table = param_info + table
    
    return full_table

In [94]:
xy_table = create_coefficient_comparison_table(J=10.0, M=0, n_max=20, n_steps=10, b=3, d=2, decimals=3)
print(xy_table)

100%|██████████| 10/10 [02:11<00:00, 13.16s/it]

Parameters: J=10.0, M=0, n_steps=10, b=3, d=2
Model: XY Model (Factorized)
+----------------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Source               |   λ(1) |   λ(2) |   λ(3) |   λ(4) |   λ(5) |   λ(6) |   λ(7) |   λ(8) |   λ(9) |   λ(10) |   λ(11) |   λ(12) |   λ(13) |   λ(14) |   λ(15) |   λ(16) |   λ(17) |   λ(18) |   λ(19) |   λ(20) |
+======================+========+========+========+========+========+========+========+========+========+=========+=========+=========+=========+=========+=========+=========+=========+=========+=========+=========+
| RG Flow              |  0.989 |  0.958 |  0.909 |  0.844 |  0.767 |  0.683 |  0.595 |  0.507 |  0.424 |   0.346 |   0.277 |   0.217 |   0.167 |   0.125 |   0.092 |   0.066 |   0.047 |   0.032 |   0.022 |   0.014 |
+----------------------+--------+--------+--------+--------+-